# OBRIGATORIA


In [1]:
%pip install pandas
%pip install matplot
%pip install mlxtend

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules, apriori
from importlib import reload
import numpy as np
import csv

plt=reload(plt)

# FILE


In [49]:
pasta = "auto"

FILE_NAME_REFACTORING = 'csv/refactorings-' + pasta +  '.csv'
FILE_NAME_COMMENTS = 'csv/comments-' + pasta + '.csv'

FILE_NAME_TABELA_AGRUPADA = pasta + '/tabela-agrupada.csv'
FILE_NAME_TABELA_MERGE = pasta + '/tabela-merge-comentarios.csv'
FILE_NAME_TABELA_COMPLETA = pasta + '/tabela-completa.csv'
FILE_NAME_TABELA_REGRAS = pasta + '/tabela-regras.csv'
FILE_NAME_TABELA_DIFERENCA_COMENTARIOS = pasta + '/tabela-diferenca-comentarios.csv'

FILE_NAME_PIVOTED = pasta + '/pivoted.csv'
FILE_NAME_QUANTIDADE_COMENTARIOS = pasta + '/quantidade-comentarios.csv'

# TRABALHANDO NOS CALCULOS

In [71]:
DF_COMMENTS = pd.read_csv(FILE_NAME_COMMENTS)
DF_COMMENTS.drop(columns=['10_HASH', '13_TYPE', '14_STARTLINE', '15_ENDLINE'], inplace=True)
DF_COMMENTS

,11_HASH_CLASSPATH,12_PARENTHASH_CLASSPATH,16_SEGMENTOS,18_ISBLOCKCOMMENT,19_ISLINECOMMENT,20_ISJAVADOCCOMMENT,21_ISORPHANCOMMENTA
0,c9c7f0e66b81a8c90246739fbf3984d0f261ae95/value...,8a5df7bfad1ac814bf6fc0a5db1842fe4874bae7/value...,5,0,0,1,0
1,c9c7f0e66b81a8c90246739fbf3984d0f261ae95/value...,8a5df7bfad1ac814bf6fc0a5db1842fe4874bae7/value...,1,0,1,0,0
2,c9c7f0e66b81a8c90246739fbf3984d0f261ae95/value...,8a5df7bfad1ac814bf6fc0a5db1842fe4874bae7/value...,4,0,0,1,0
3,c9c7f0e66b81a8c90246739fbf3984d0f261ae95/value...,8a5df7bfad1ac814bf6fc0a5db1842fe4874bae7/value...,1,0,0,1,0
4,c9c7f0e66b81a8c90246739fbf3984d0f261ae95/value...,8a5df7bfad1ac814bf6fc0a5db1842fe4874bae7/value...,1,0,0,1,0
...,...,...,...,...,...,...,...
2837228,10da2957f3dd7316c40bba00b16ad0c6244d53d5/value...,040372c5f7473e961ea77c54c0ad14c822af6bac/value...,1,0,1,0,1
2837229,10da2957f3dd7316c40bba00b16ad0c6244d53d5/value...,040372c5f7473e961ea77c54c0ad14c822af6bac/value...,1,0,1,0,1
2837230,10da2957f3dd7316c40bba00b16ad0c6244d53d5/value...,040372c5f7473e961ea77c54c0ad14c822af6bac/value...,1,0,1,0,1
2837231,10da2957f3dd7316c40bba00b16ad0c6244d53d5/value...,040372c5f7473e961ea77c54c0ad14c822af6bac/value...,1,0,1,0,0


In [72]:
# Agrupamos pelo '11_HASH_CLASSPATH' e '12_PARENTHASH_CLASSPATH' e somamos as outras colunas
agrupado = DF_COMMENTS.groupby(["11_HASH_CLASSPATH", "12_PARENTHASH_CLASSPATH"]).sum().reset_index()

# Renomeando as colunas para os novos nomes
agrupado.rename(columns={
    "16_SEGMENTOS": "QNT_SEGMENTOS",
    "18_ISBLOCKCOMMENT": "QNT_ISBLOCKCOMMENT",
    "19_ISLINECOMMENT": "QTN_ISLINECOMMENT",
    "20_ISJAVADOCCOMMENT": "QTN_ISJAVADOCCOMMENT",
    "21_ISORPHANCOMMENTA": "QTN_ISORPHANCOMMENTA"

}, inplace=True) 



agrupado.to_csv(FILE_NAME_TABELA_AGRUPADA, index=False)
agrupado

,11_HASH_CLASSPATH,12_PARENTHASH_CLASSPATH,QNT_SEGMENTOS,QNT_ISBLOCKCOMMENT,QTN_ISLINECOMMENT,QTN_ISJAVADOCCOMMENT,QTN_ISORPHANCOMMENTA
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,57,0,2,9,0
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,27,0,5,4,2
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,161,0,10,28,2
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,164,0,7,21,3
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,30,0,0,5,0
...,...,...,...,...,...,...,...
268363,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,12,1,2,1,3
268364,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,6,0,0,1,0
268365,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,6,0,0,1,0
268366,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,6,0,1,1,1


In [73]:
# Juntando o DataFrame consigo mesmo para calcular as diferenças

merged_df = pd.merge(agrupado, agrupado, right_on="11_HASH_CLASSPATH", left_on="12_PARENTHASH_CLASSPATH", suffixes=('', '_PARENTE'))

merged_df.drop(columns=['11_HASH_CLASSPATH_PARENTE', '12_PARENTHASH_CLASSPATH_PARENTE'], inplace=True)

merged_df.to_csv(FILE_NAME_TABELA_MERGE, index=False)

merged_df

,11_HASH_CLASSPATH,12_PARENTHASH_CLASSPATH,QNT_SEGMENTOS,QNT_ISBLOCKCOMMENT,QTN_ISLINECOMMENT,QTN_ISJAVADOCCOMMENT,QTN_ISORPHANCOMMENTA,QNT_SEGMENTOS_PARENTE,QNT_ISBLOCKCOMMENT_PARENTE,QTN_ISLINECOMMENT_PARENTE,QTN_ISJAVADOCCOMMENT_PARENTE,QTN_ISORPHANCOMMENTA_PARENTE
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,57,0,2,9,0,57,0,2,9,0
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,27,0,5,4,2,27,0,5,4,2
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,161,0,10,28,2,161,0,10,28,2
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,164,0,7,21,3,164,0,7,21,3
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,30,0,0,5,0,30,0,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
251559,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,12,1,2,1,3,12,1,2,1,2
251560,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,6,0,0,1,0,9,2,1,1,0
251561,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,6,0,0,1,0,6,0,0,1,0
251562,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,6,0,1,1,1,5,0,0,1,0


# CALCULO DIFERENCAS

In [74]:

# Calculando as diferenças
merged_df["DIFERENCA_QNT_ISBLOCKCOMMENT"] = merged_df["QNT_ISBLOCKCOMMENT"] - merged_df["QNT_ISBLOCKCOMMENT_PARENTE"]
merged_df["DIFERENCA_QTN_ISLINECOMMENT"] = merged_df["QTN_ISLINECOMMENT"] - merged_df["QTN_ISLINECOMMENT_PARENTE"]
merged_df["DIFERENCA_QTN_ISJAVADOCCOMMENT"] = merged_df["QTN_ISJAVADOCCOMMENT"] - merged_df["QTN_ISJAVADOCCOMMENT_PARENTE"]
merged_df["DIFERENCA_QTN_ISORPHANCOMMENTA"] = merged_df["QTN_ISORPHANCOMMENTA"] - merged_df["QTN_ISORPHANCOMMENTA_PARENTE"]
merged_df["DIFERENCA_QNT_SEGMENTOS"] =  merged_df["QNT_SEGMENTOS"] - merged_df["QNT_SEGMENTOS_PARENTE"]

merged_df

,11_HASH_CLASSPATH,12_PARENTHASH_CLASSPATH,QNT_SEGMENTOS,QNT_ISBLOCKCOMMENT,QTN_ISLINECOMMENT,QTN_ISJAVADOCCOMMENT,QTN_ISORPHANCOMMENTA,QNT_SEGMENTOS_PARENTE,QNT_ISBLOCKCOMMENT_PARENTE,QTN_ISLINECOMMENT_PARENTE,QTN_ISJAVADOCCOMMENT_PARENTE,QTN_ISORPHANCOMMENTA_PARENTE,DIFERENCA_QNT_ISBLOCKCOMMENT,DIFERENCA_QTN_ISLINECOMMENT,DIFERENCA_QTN_ISJAVADOCCOMMENT,DIFERENCA_QTN_ISORPHANCOMMENTA,DIFERENCA_QNT_SEGMENTOS
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,57,0,2,9,0,57,0,2,9,0,0,0,0,0,0
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,27,0,5,4,2,27,0,5,4,2,0,0,0,0,0
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,161,0,10,28,2,161,0,10,28,2,0,0,0,0,0
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,164,0,7,21,3,164,0,7,21,3,0,0,0,0,0
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,30,0,0,5,0,30,0,0,5,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251559,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,12,1,2,1,3,12,1,2,1,2,0,0,0,1,0
251560,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,6,0,0,1,0,9,2,1,1,0,-2,-1,0,0,-3
251561,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,6,0,0,1,0,6,0,0,1,0,0,0,0,0,0
251562,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,6,0,1,1,1,5,0,0,1,0,0,1,0,1,1


In [75]:
# Selecionando as colunas desejadas
novo_df_com_diferencas = merged_df[[
    "11_HASH_CLASSPATH", "12_PARENTHASH_CLASSPATH",
    "QNT_ISBLOCKCOMMENT", "QTN_ISLINECOMMENT", "QTN_ISJAVADOCCOMMENT", "QTN_ISORPHANCOMMENTA",
    "DIFERENCA_QNT_ISBLOCKCOMMENT", "DIFERENCA_QTN_ISLINECOMMENT","DIFERENCA_QNT_SEGMENTOS",
    "DIFERENCA_QTN_ISJAVADOCCOMMENT", "DIFERENCA_QTN_ISORPHANCOMMENTA"
]]

# Removendo as linhas onde não há correspondência para 12_PARENTHASH_CLASSPATH
# novo_df_com_diferencas.dropna(inplace=True)

# Removendo as linhas onde 12_PARENTHASH_CLASSPATH começa com '[' PQ TEM AMIS DE UM PAI
# novo_df_com_diferencas = novo_df_com_diferencas[~novo_df_com_diferencas['12_PARENTHASH_CLASSPATH'].str.startswith('[')]

# Preenchendo os valores NaN com 0
# novo_df_com_diferencas.fillna(0, inplace=True)

novo_df_com_diferencas.drop(columns=['12_PARENTHASH_CLASSPATH', 'QNT_ISBLOCKCOMMENT',  'QTN_ISLINECOMMENT', 'QTN_ISJAVADOCCOMMENT', 'QTN_ISORPHANCOMMENTA'], inplace=True)
novo_df_com_diferencas.to_csv(FILE_NAME_TABELA_DIFERENCA_COMENTARIOS, index=False)
novo_df_com_diferencas

C:\Users\kleit\AppData\Local\Temp\ipykernel_17308\1231487294.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  novo_df_com_diferencas.drop(columns=['12_PARENTHASH_CLASSPATH', 'QNT_ISBLOCKCOMMENT',  'QTN_ISLINECOMMENT', 'QTN_ISJAVADOCCOMMENT', 'QTN_ISORPHANCOMMENTA'], inplace=True)


,11_HASH_CLASSPATH,DIFERENCA_QNT_ISBLOCKCOMMENT,DIFERENCA_QTN_ISLINECOMMENT,DIFERENCA_QNT_SEGMENTOS,DIFERENCA_QTN_ISJAVADOCCOMMENT,DIFERENCA_QTN_ISORPHANCOMMENTA
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,0,0,0
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,0,0,0
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,0,0,0
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,0,0,0
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,0,0,0
...,...,...,...,...,...,...
251559,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0,0,0,0,1
251560,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,-2,-1,-3,0,0
251561,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0,0,0,0,0
251562,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0,1,1,0,1


In [76]:
df = novo_df_com_diferencas.copy()
df

,11_HASH_CLASSPATH,DIFERENCA_QNT_ISBLOCKCOMMENT,DIFERENCA_QTN_ISLINECOMMENT,DIFERENCA_QNT_SEGMENTOS,DIFERENCA_QTN_ISJAVADOCCOMMENT,DIFERENCA_QTN_ISORPHANCOMMENTA
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,0,0,0
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,0,0,0
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,0,0,0
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,0,0,0
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,0,0,0
...,...,...,...,...,...,...
251559,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0,0,0,0,1
251560,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,-2,-1,-3,0,0
251561,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0,0,0,0,0
251562,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0,1,1,0,1


In [77]:
# Criar uma nova coluna 'COMMENTS_CHANGE' com valores padrão como None
df['DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE'] = None
df['DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou'] = None
df['DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu'] = None
df['DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve'] = None

# Atualizar a coluna 'DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QNT_ISBLOCKCOMMENT'] > 0, 'DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou'] = 1
df.loc[df['DIFERENCA_QNT_ISBLOCKCOMMENT'] < 0, 'DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu'] = 1
df.loc[df['DIFERENCA_QNT_ISBLOCKCOMMENT'] == 0, 'DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve'] = 1

# Atualizar a coluna 'DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QNT_ISBLOCKCOMMENT'] > 0, 'DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE'] = df['DIFERENCA_QNT_ISBLOCKCOMMENT']
df.loc[df['DIFERENCA_QNT_ISBLOCKCOMMENT'] < 0, 'DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE'] = df['DIFERENCA_QNT_ISBLOCKCOMMENT']
df.loc[df['DIFERENCA_QNT_ISBLOCKCOMMENT'] == 0, 'DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE'] = 0

df['DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE'].fillna(0, inplace=True)
df['DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou'].fillna(0, inplace=True)
df['DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu'].fillna(0, inplace=True)
df['DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve'].fillna(0, inplace=True)


# Definir faixas de variação
faixas = {
    'DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_1_5': ((df['DIFERENCA_QNT_ISBLOCKCOMMENT'] >= 1) & (df['DIFERENCA_QNT_ISBLOCKCOMMENT'] < 5)),
    'DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_5_10': ((df['DIFERENCA_QNT_ISBLOCKCOMMENT'] >= 5) & (df['DIFERENCA_QNT_ISBLOCKCOMMENT'] < 10)),
    'DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_10_15': ((df['DIFERENCA_QNT_ISBLOCKCOMMENT'] >= 10) & (df['DIFERENCA_QNT_ISBLOCKCOMMENT'] < 15)),
    'DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_15': (df['DIFERENCA_QNT_ISBLOCKCOMMENT'] >= 15),
    'DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_1_5': ((df['DIFERENCA_QNT_ISBLOCKCOMMENT'] <= -1) & (df['DIFERENCA_QNT_ISBLOCKCOMMENT'] > -5)),
    'DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_5_10': ((df['DIFERENCA_QNT_ISBLOCKCOMMENT'] <= -5) & (df['DIFERENCA_QNT_ISBLOCKCOMMENT'] > -10)),
    'DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_10_15': ((df['DIFERENCA_QNT_ISBLOCKCOMMENT'] <= -10) & (df['DIFERENCA_QNT_ISBLOCKCOMMENT'] > -15)),
    'DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_15': (df['DIFERENCA_QNT_ISBLOCKCOMMENT'] <= -15)
}

# Criar colunas e preencher com base nas faixas
for coluna, condicao in faixas.items():
    df[coluna] = condicao.astype(int)

df.drop(columns=['DIFERENCA_QNT_ISBLOCKCOMMENT' , 'DIFERENCA_QNT_ISBLOCKCOMMENT_CHANGE'], inplace=True)

df

,11_HASH_CLASSPATH,DIFERENCA_QTN_ISLINECOMMENT,DIFERENCA_QNT_SEGMENTOS,DIFERENCA_QTN_ISJAVADOCCOMMENT,DIFERENCA_QTN_ISORPHANCOMMENTA,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu,DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_1_5,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_5_10,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_10_15,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_15,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_1_5,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_5_10,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_10_15,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_15
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251559,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
251560,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,-1,-3,0,0,0,1,0,0,0,0,0,1,0,0,0
251561,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
251562,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0


In [78]:
# Criar uma nova coluna 'COMMENTS_CHANGE' com valores padrão como None
df['DIFERENCA_QTN_ISLINECOMMENT_CHANGE'] = None
df['DIFERENCA_QTN_ISLINECOMMENT_Aumentou'] = None
df['DIFERENCA_QTN_ISLINECOMMENT_Diminuiu'] = None
df['DIFERENCA_QTN_ISLINECOMMENT_Manteve'] = None

# Atualizar a coluna 'DIFERENCA_QTN_ISLINECOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_ISLINECOMMENT'] > 0, 'DIFERENCA_QTN_ISLINECOMMENT_Aumentou'] = 1
df.loc[df['DIFERENCA_QTN_ISLINECOMMENT'] < 0, 'DIFERENCA_QTN_ISLINECOMMENT_Diminuiu'] = 1
df.loc[df['DIFERENCA_QTN_ISLINECOMMENT'] == 0, 'DIFERENCA_QTN_ISLINECOMMENT_Manteve'] = 1

# Atualizar a coluna 'DIFERENCA_QTN_ISLINECOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_ISLINECOMMENT'] > 0, 'DIFERENCA_QTN_ISLINECOMMENT_CHANGE'] = df['DIFERENCA_QTN_ISLINECOMMENT']
df.loc[df['DIFERENCA_QTN_ISLINECOMMENT'] < 0, 'DIFERENCA_QTN_ISLINECOMMENT_CHANGE'] = df['DIFERENCA_QTN_ISLINECOMMENT']
df.loc[df['DIFERENCA_QTN_ISLINECOMMENT'] == 0, 'DIFERENCA_QTN_ISLINECOMMENT_CHANGE'] = 0

df['DIFERENCA_QTN_ISLINECOMMENT_CHANGE'].fillna(0, inplace=True)
df['DIFERENCA_QTN_ISLINECOMMENT_Aumentou'].fillna(0, inplace=True)
df['DIFERENCA_QTN_ISLINECOMMENT_Diminuiu'].fillna(0, inplace=True)
df['DIFERENCA_QTN_ISLINECOMMENT_Manteve'].fillna(0, inplace=True)

# Definir faixas de variação
faixas = {
    'DIFERENCA_QTN_ISLINECOMMENT_Aumentou_1_5': ((df['DIFERENCA_QTN_ISLINECOMMENT'] >= 1) & (df['DIFERENCA_QTN_ISLINECOMMENT'] < 5)),
    'DIFERENCA_QTN_ISLINECOMMENT_Aumentou_5_10': ((df['DIFERENCA_QTN_ISLINECOMMENT'] >= 5) & (df['DIFERENCA_QTN_ISLINECOMMENT'] < 10)),
    'DIFERENCA_QTN_ISLINECOMMENT_Aumentou_10_15': ((df['DIFERENCA_QTN_ISLINECOMMENT'] >= 10) & (df['DIFERENCA_QTN_ISLINECOMMENT'] < 15)),
    'DIFERENCA_QTN_ISLINECOMMENT_Aumentou_15': (df['DIFERENCA_QTN_ISLINECOMMENT'] >= 15),
    'DIFERENCA_QTN_ISLINECOMMENT_Diminuiu_1_5': ((df['DIFERENCA_QTN_ISLINECOMMENT'] <= -1) & (df['DIFERENCA_QTN_ISLINECOMMENT'] > -5)),
    'DIFERENCA_QTN_ISLINECOMMENT_Diminuiu_5_10': ((df['DIFERENCA_QTN_ISLINECOMMENT'] <= -5) & (df['DIFERENCA_QTN_ISLINECOMMENT'] > -10)),
    'DIFERENCA_QTN_ISLINECOMMENT_Diminuiu_10_15': ((df['DIFERENCA_QTN_ISLINECOMMENT'] <= -10) & (df['DIFERENCA_QTN_ISLINECOMMENT'] > -15)),
    'DIFERENCA_QTN_ISLINECOMMENT_Diminuiu_15': (df['DIFERENCA_QTN_ISLINECOMMENT'] <= -15)
}

# Criar colunas e preencher com base nas faixas
for coluna, condicao in faixas.items():
    df[coluna] = condicao.astype(int)

df.drop(columns=['DIFERENCA_QTN_ISLINECOMMENT', 'DIFERENCA_QTN_ISLINECOMMENT_CHANGE'], inplace=True)

df

,11_HASH_CLASSPATH,DIFERENCA_QNT_SEGMENTOS,DIFERENCA_QTN_ISJAVADOCCOMMENT,DIFERENCA_QTN_ISORPHANCOMMENTA,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu,DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_1_5,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_5_10,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_10_15,...,DIFERENCA_QTN_ISLINECOMMENT_Diminuiu,DIFERENCA_QTN_ISLINECOMMENT_Manteve,DIFERENCA_QTN_ISLINECOMMENT_Aumentou_1_5,DIFERENCA_QTN_ISLINECOMMENT_Aumentou_5_10,DIFERENCA_QTN_ISLINECOMMENT_Aumentou_10_15,DIFERENCA_QTN_ISLINECOMMENT_Aumentou_15,DIFERENCA_QTN_ISLINECOMMENT_Diminuiu_1_5,DIFERENCA_QTN_ISLINECOMMENT_Diminuiu_5_10,DIFERENCA_QTN_ISLINECOMMENT_Diminuiu_10_15,DIFERENCA_QTN_ISLINECOMMENT_Diminuiu_15
5302,03df7e1f627d3b6a7e66d0cbc89f8b4d9bd0cfaa/src-s...,4,0,1,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
14576,0c7c4ab7a304173355caf518e451b7b1cd3eadc3/facto...,16,1,2,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
15561,0d2ba512f532f9b69111e2b940cb721bf5f42836/value...,8,0,4,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
17909,0f97cad6dd0cabce0ebf97354dfb33efc498f253/value...,15,0,1,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
24602,1439990cc6a01156c9cbaa98a087cdb03a129fca/value...,-1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
29884,173c6ac0cb10211fd37672292805702190cde2bc/commo...,-8,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
31651,196c8100d43b5212c31a83912d66791c5a64525b/value...,57,0,52,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
42499,24407feff070fc57ad9bfa4527659da5c5bc44d9/value...,4,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
44875,26f073f8371577da8237631767ad127c41bbb462/commo...,0,0,-1,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
57283,324470ba28e85482c7b359a3679601583323d9ab/value...,5,0,3,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [79]:
# Criar uma nova coluna 'COMMENTS_CHANGE' com valores padrão como None
df['DIFERENCA_QNT_SEGMENTOS_CHANGE'] = None
df['DIFERENCA_QNT_SEGMENTOS_Aumentou'] = None
df['DIFERENCA_QNT_SEGMENTOS_Diminuiu'] = None
df['DIFERENCA_QNT_SEGMENTOS_Manteve'] = None

# Atualizar a coluna 'DIFERENCA_QNT_SEGMENTOS_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QNT_SEGMENTOS'] > 0, 'DIFERENCA_QNT_SEGMENTOS_Aumentou'] = 1
df.loc[df['DIFERENCA_QNT_SEGMENTOS'] < 0, 'DIFERENCA_QNT_SEGMENTOS_Diminuiu'] = 1
df.loc[df['DIFERENCA_QNT_SEGMENTOS'] == 0, 'DIFERENCA_QNT_SEGMENTOS_Manteve'] = 1

# Atualizar a coluna 'DIFERENCA_QNT_SEGMENTOS_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QNT_SEGMENTOS'] > 0, 'DIFERENCA_QNT_SEGMENTOS_CHANGE'] = df['DIFERENCA_QNT_SEGMENTOS']
df.loc[df['DIFERENCA_QNT_SEGMENTOS'] < 0, 'DIFERENCA_QNT_SEGMENTOS_CHANGE'] = df['DIFERENCA_QNT_SEGMENTOS']
df.loc[df['DIFERENCA_QNT_SEGMENTOS'] == 0, 'DIFERENCA_QNT_SEGMENTOS_CHANGE'] = 0

df['DIFERENCA_QNT_SEGMENTOS_CHANGE'].fillna(0, inplace=True)
df['DIFERENCA_QNT_SEGMENTOS_Aumentou'].fillna(0, inplace=True)
df['DIFERENCA_QNT_SEGMENTOS_Diminuiu'].fillna(0, inplace=True)
df['DIFERENCA_QNT_SEGMENTOS_Manteve'].fillna(0, inplace=True)

faixas = {
    'DIFERENCA_QNT_SEGMENTOS_Aumentou_1_5': ((df['DIFERENCA_QNT_SEGMENTOS'] >= 1) & (df['DIFERENCA_QNT_SEGMENTOS'] < 5)),
    'DIFERENCA_QNT_SEGMENTOS_Aumentou_5_10': ((df['DIFERENCA_QNT_SEGMENTOS'] >= 5) & (df['DIFERENCA_QNT_SEGMENTOS'] < 10)),
    'DIFERENCA_QNT_SEGMENTOS_Aumentou_10_15': ((df['DIFERENCA_QNT_SEGMENTOS'] >= 10) & (df['DIFERENCA_QNT_SEGMENTOS'] < 15)),
    'DIFERENCA_QNT_SEGMENTOS_Aumentou_15': (df['DIFERENCA_QNT_SEGMENTOS'] >= 15),
    'DIFERENCA_QNT_SEGMENTOS_Diminuiu_1_5': ((df['DIFERENCA_QNT_SEGMENTOS'] <= -1) & (df['DIFERENCA_QNT_SEGMENTOS'] > -5)),
    'DIFERENCA_QNT_SEGMENTOS_Diminuiu_5_10': ((df['DIFERENCA_QNT_SEGMENTOS'] <= -5) & (df['DIFERENCA_QNT_SEGMENTOS'] > -10)),
    'DIFERENCA_QNT_SEGMENTOS_Diminuiu_10_15': ((df['DIFERENCA_QNT_SEGMENTOS'] <= -10) & (df['DIFERENCA_QNT_SEGMENTOS'] > -15)),
    'DIFERENCA_QNT_SEGMENTOS_Diminuiu_15': (df['DIFERENCA_QNT_SEGMENTOS'] <= -15)
}

# Criar colunas e preencher com base nas faixas
for coluna, condicao in faixas.items():
    df[coluna] = condicao.astype(int)

df.drop(columns=['DIFERENCA_QNT_SEGMENTOS', 'DIFERENCA_QNT_SEGMENTOS_CHANGE'], inplace=True)

df

C:\Users\kleit\AppData\Local\Temp\ipykernel_17308\2909379500.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QNT_SEGMENTOS_CHANGE'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_17308\2909379500.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QNT_SEGMENTOS_Aumentou'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_17308\2909379500.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

,11_HASH_CLASSPATH,DIFERENCA_QTN_ISJAVADOCCOMMENT,DIFERENCA_QTN_ISORPHANCOMMENTA,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu,DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_1_5,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_5_10,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_10_15,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_15,...,DIFERENCA_QNT_SEGMENTOS_Diminuiu,DIFERENCA_QNT_SEGMENTOS_Manteve,DIFERENCA_QNT_SEGMENTOS_Aumentou_1_5,DIFERENCA_QNT_SEGMENTOS_Aumentou_5_10,DIFERENCA_QNT_SEGMENTOS_Aumentou_10_15,DIFERENCA_QNT_SEGMENTOS_Aumentou_15,DIFERENCA_QNT_SEGMENTOS_Diminuiu_1_5,DIFERENCA_QNT_SEGMENTOS_Diminuiu_5_10,DIFERENCA_QNT_SEGMENTOS_Diminuiu_10_15,DIFERENCA_QNT_SEGMENTOS_Diminuiu_15
5302,03df7e1f627d3b6a7e66d0cbc89f8b4d9bd0cfaa/src-s...,0,1,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
14576,0c7c4ab7a304173355caf518e451b7b1cd3eadc3/facto...,1,2,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
15561,0d2ba512f532f9b69111e2b940cb721bf5f42836/value...,0,4,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
17909,0f97cad6dd0cabce0ebf97354dfb33efc498f253/value...,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
24602,1439990cc6a01156c9cbaa98a087cdb03a129fca/value...,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
29884,173c6ac0cb10211fd37672292805702190cde2bc/commo...,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
31651,196c8100d43b5212c31a83912d66791c5a64525b/value...,0,52,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
42499,24407feff070fc57ad9bfa4527659da5c5bc44d9/value...,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
44875,26f073f8371577da8237631767ad127c41bbb462/commo...,0,-1,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
57283,324470ba28e85482c7b359a3679601583323d9ab/value...,0,3,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [80]:
# Criar uma nova coluna 'COMMENTS_CHANGE' com valores padrão como None
df['DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE'] = None
df['DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou'] = None
df['DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu'] = None
df['DIFERENCA_QTN_ISJAVADOCCOMMENT_Manteve'] = None

# Atualizar a coluna 'DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] > 0, 'DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou'] = 1
df.loc[df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] < 0, 'DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu'] = 1
df.loc[df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] == 0, 'DIFERENCA_QTN_ISJAVADOCCOMMENT_Manteve'] = 1

# Atualizar a coluna 'DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] > 0, 'DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE'] = df['DIFERENCA_QTN_ISJAVADOCCOMMENT']
df.loc[df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] < 0, 'DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE'] = df['DIFERENCA_QTN_ISJAVADOCCOMMENT']
df.loc[df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] == 0, 'DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE'] = 0

df['DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE'].fillna(0, inplace=True)
df['DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou'].fillna(0, inplace=True)
df['DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu'].fillna(0, inplace=True)
df['DIFERENCA_QTN_ISJAVADOCCOMMENT_Manteve'].fillna(0, inplace=True)

faixas = {
    'DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou_1_5': ((df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] >= 1) & (df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] < 5)),
    'DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou_5_10': ((df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] >= 5) & (df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] < 10)),
    'DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou_10_15': ((df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] >= 10) & (df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] < 15)),
    'DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou_15': (df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] >= 15),
    'DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu_1_5': ((df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] <= -1) & (df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] > -5)),
    'DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu_5_10': ((df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] <= -5) & (df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] > -10)),
    'DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu_10_15': ((df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] <= -10) & (df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] > -15)),
    'DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu_15': (df['DIFERENCA_QTN_ISJAVADOCCOMMENT'] <= -15)
}

# Criar colunas e preencher com base nas faixas
for coluna, condicao in faixas.items():
    df[coluna] = condicao.astype(int)

df.drop(columns=['DIFERENCA_QTN_ISJAVADOCCOMMENT', 'DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE'], inplace=True)

df

C:\Users\kleit\AppData\Local\Temp\ipykernel_17308\3596986029.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QTN_ISJAVADOCCOMMENT_CHANGE'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_17308\3596986029.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_17308\3596986029.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

,11_HASH_CLASSPATH,DIFERENCA_QTN_ISORPHANCOMMENTA,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu,DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_1_5,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_5_10,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_10_15,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_15,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_1_5,...,DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu,DIFERENCA_QTN_ISJAVADOCCOMMENT_Manteve,DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou_1_5,DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou_5_10,DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou_10_15,DIFERENCA_QTN_ISJAVADOCCOMMENT_Aumentou_15,DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu_1_5,DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu_5_10,DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu_10_15,DIFERENCA_QTN_ISJAVADOCCOMMENT_Diminuiu_15
5302,03df7e1f627d3b6a7e66d0cbc89f8b4d9bd0cfaa/src-s...,1,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
14576,0c7c4ab7a304173355caf518e451b7b1cd3eadc3/facto...,2,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
15561,0d2ba512f532f9b69111e2b940cb721bf5f42836/value...,4,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
17909,0f97cad6dd0cabce0ebf97354dfb33efc498f253/value...,1,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
24602,1439990cc6a01156c9cbaa98a087cdb03a129fca/value...,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
29884,173c6ac0cb10211fd37672292805702190cde2bc/commo...,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
31651,196c8100d43b5212c31a83912d66791c5a64525b/value...,52,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
42499,24407feff070fc57ad9bfa4527659da5c5bc44d9/value...,0,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
44875,26f073f8371577da8237631767ad127c41bbb462/commo...,-1,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
57283,324470ba28e85482c7b359a3679601583323d9ab/value...,3,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [81]:

# Criar uma nova coluna 'COMMENTS_CHANGE' com valores padrão como None
df['DIFERENCA_QTN_ISORPHANCOMMENTA_CHANGE'] = None
df['DIFERENCA_QTN_ISORPHANCOMMENTA_Aumentou'] = None
df['DIFERENCA_QTN_ISORPHANCOMMENTA_Diminuiu'] = None
df['DIFERENCA_QTN_ISORPHANCOMMENTA_Manteve'] = None

# Atualizar a coluna 'DIFERENCA_QTN_ISORPHANCOMMENTA_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_ISORPHANCOMMENTA'] > 0, 'DIFERENCA_QTN_ISORPHANCOMMENTA_Aumentou'] = 1
df.loc[df['DIFERENCA_QTN_ISORPHANCOMMENTA'] < 0, 'DIFERENCA_QTN_ISORPHANCOMMENTA_Diminuiu'] = 1
df.loc[df['DIFERENCA_QTN_ISORPHANCOMMENTA'] == 0, 'DIFERENCA_QTN_ISORPHANCOMMENTA_Manteve'] = 1

# Atualizar a coluna 'DIFERENCA_QTN_ISORPHANCOMMENTA_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_ISORPHANCOMMENTA'] > 0, 'DIFERENCA_QTN_ISORPHANCOMMENTA_CHANGE'] = df['DIFERENCA_QTN_ISORPHANCOMMENTA']
df.loc[df['DIFERENCA_QTN_ISORPHANCOMMENTA'] < 0, 'DIFERENCA_QTN_ISORPHANCOMMENTA_CHANGE'] = df['DIFERENCA_QTN_ISORPHANCOMMENTA']
df.loc[df['DIFERENCA_QTN_ISORPHANCOMMENTA'] == 0, 'DIFERENCA_QTN_ISORPHANCOMMENTA_CHANGE'] = 0

df['DIFERENCA_QTN_ISORPHANCOMMENTA_CHANGE'].fillna(0, inplace=True)
df['DIFERENCA_QTN_ISORPHANCOMMENTA_Aumentou'].fillna(0, inplace=True)
df['DIFERENCA_QTN_ISORPHANCOMMENTA_Diminuiu'].fillna(0, inplace=True)
df['DIFERENCA_QTN_ISORPHANCOMMENTA_Manteve'].fillna(0, inplace=True)

faixas = {
    'DIFERENCA_QTN_ISORPHANCOMMENTA_Aumentou_1_5': ((df['DIFERENCA_QTN_ISORPHANCOMMENTA'] >= 1) & (df['DIFERENCA_QTN_ISORPHANCOMMENTA'] < 5)),
    'DIFERENCA_QTN_ISORPHANCOMMENTA_Aumentou_5_10': ((df['DIFERENCA_QTN_ISORPHANCOMMENTA'] >= 5) & (df['DIFERENCA_QTN_ISORPHANCOMMENTA'] < 10)),
    'DIFERENCA_QTN_ISORPHANCOMMENTA_Aumentou_10_15': ((df['DIFERENCA_QTN_ISORPHANCOMMENTA'] >= 10) & (df['DIFERENCA_QTN_ISORPHANCOMMENTA'] < 15)),
    'DIFERENCA_QTN_ISORPHANCOMMENTA_Aumentou_15': (df['DIFERENCA_QTN_ISORPHANCOMMENTA'] >= 15),
    'DIFERENCA_QTN_ISORPHANCOMMENTA_Diminuiu_1_5': ((df['DIFERENCA_QTN_ISORPHANCOMMENTA'] <= -1) & (df['DIFERENCA_QTN_ISORPHANCOMMENTA'] > -5)),
    'DIFERENCA_QTN_ISORPHANCOMMENTA_Diminuiu_5_10': ((df['DIFERENCA_QTN_ISORPHANCOMMENTA'] <= -5) & (df['DIFERENCA_QTN_ISORPHANCOMMENTA'] > -10)),
    'DIFERENCA_QTN_ISORPHANCOMMENTA_Diminuiu_10_15': ((df['DIFERENCA_QTN_ISORPHANCOMMENTA'] <= -10) & (df['DIFERENCA_QTN_ISORPHANCOMMENTA'] > -15)),
    'DIFERENCA_QTN_ISORPHANCOMMENTA_Diminuiu_15': (df['DIFERENCA_QTN_ISORPHANCOMMENTA'] <= -15)
}

# Criar colunas e preencher com base nas faixas
for coluna, condicao in faixas.items():
    df[coluna] = condicao.astype(int)

df.drop(columns=['DIFERENCA_QTN_ISORPHANCOMMENTA', 'DIFERENCA_QTN_ISORPHANCOMMENTA_CHANGE'], inplace=True)

df

df.to_csv(FILE_NAME_QUANTIDADE_COMENTARIOS, index=False)


C:\Users\kleit\AppData\Local\Temp\ipykernel_17308\3840234128.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QTN_ISORPHANCOMMENTA_CHANGE'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_17308\3840234128.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QTN_ISORPHANCOMMENTA_Aumentou'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_17308\3840234128.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

# Refactorings

In [82]:
DF_REFACTORING = pd.read_csv(FILE_NAME_REFACTORING)
DF_REFACTORING.drop(columns=['12_MOVEDTOPATHCLASS', '14_OCORRIDO'], inplace=True)
DF_REFACTORING

,11_HASH_CLASSPATH,13_REFACTORINGTYPE
0,02116e084752ce2247275910c81b89a3767362bb/value...,CHANGE_CLASS_ACCESS_MODIFIER
1,02116e084752ce2247275910c81b89a3767362bb/value...,REMOVE_METHOD_ANNOTATION
2,02116e084752ce2247275910c81b89a3767362bb/value...,CHANGE_VARIABLE_TYPE
3,02116e084752ce2247275910c81b89a3767362bb/value...,CHANGE_VARIABLE_TYPE
4,02116e084752ce2247275910c81b89a3767362bb/value...,CHANGE_VARIABLE_TYPE
...,...,...
5464,46e6d85ef3abb8dd6d20e9b820ecb7022529f26e/value...,EXTRACT_OPERATION
5465,46e6d85ef3abb8dd6d20e9b820ecb7022529f26e/value...,CHANGE_VARIABLE_TYPE
5466,46e6d85ef3abb8dd6d20e9b820ecb7022529f26e/value...,PARAMETERIZE_VARIABLE
5467,46e6d85ef3abb8dd6d20e9b820ecb7022529f26e/value...,ADD_PARAMETER


In [83]:
pivoted = DF_REFACTORING.pivot_table(index='11_HASH_CLASSPATH', columns='13_REFACTORINGTYPE', aggfunc='size', fill_value=0)
pivoted.to_csv(FILE_NAME_PIVOTED, index=True)
pivoted

13_REFACTORINGTYPE,ADD_ATTRIBUTE_ANNOTATION,ADD_ATTRIBUTE_MODIFIER,ADD_CLASS_ANNOTATION,ADD_CLASS_MODIFIER,ADD_METHOD_ANNOTATION,ADD_METHOD_MODIFIER,ADD_PARAMETER,ADD_PARAMETER_ANNOTATION,ADD_PARAMETER_MODIFIER,ADD_THROWN_EXCEPTION_TYPE,...,REORDER_PARAMETER,REPLACE_ANONYMOUS_WITH_LAMBDA,REPLACE_ATTRIBUTE_WITH_VARIABLE,REPLACE_LOOP_WITH_PIPELINE,REPLACE_VARIABLE_WITH_ATTRIBUTE,SPLIT_ATTRIBUTE,SPLIT_CONDITIONAL,SPLIT_OPERATION,SPLIT_PARAMETER,SPLIT_VARIABLE
11_HASH_CLASSPATH,,,,,,,,,,,,,,,,,,,,,
0076afb3825bd4d3b7ab77310855c2cc4a05813e/value/src/main/java/com/google/auto/value/processor/AutoValueOrOneOfProcessor.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0076afb3825bd4d3b7ab77310855c2cc4a05813e/value/src/main/java/com/google/auto/value/processor/AutoValueOrOneOfTemplateVars.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0076afb3825bd4d3b7ab77310855c2cc4a05813e/value/src/main/java/com/google/auto/value/processor/AutoValueProcessor.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0076afb3825bd4d3b7ab77310855c2cc4a05813e/value/src/main/java/com/google/auto/value/processor/AutoValueTemplateVars.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0076afb3825bd4d3b7ab77310855c2cc4a05813e/value/src/main/java/com/google/auto/value/processor/BuilderSpec.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff523aeab1ec8f5f3ad821e2d75bd3d336dd8096/factory/src/main/java/com/google/auto/factory/processor/FactoryDescriptor.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ff523aeab1ec8f5f3ad821e2d75bd3d336dd8096/factory/src/main/java/com/google/auto/factory/processor/FactoryWriter.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ff523aeab1ec8f5f3ad821e2d75bd3d336dd8096/factory/src/main/java/com/google/auto/factory/processor/Key.java,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
pivoted

13_REFACTORINGTYPE,ADD_ATTRIBUTE_ANNOTATION,ADD_ATTRIBUTE_MODIFIER,ADD_CLASS_ANNOTATION,ADD_CLASS_MODIFIER,ADD_METHOD_ANNOTATION,ADD_METHOD_MODIFIER,ADD_PARAMETER,ADD_PARAMETER_ANNOTATION,ADD_PARAMETER_MODIFIER,ADD_THROWN_EXCEPTION_TYPE,...,REORDER_PARAMETER,REPLACE_ANONYMOUS_WITH_LAMBDA,REPLACE_ATTRIBUTE_WITH_VARIABLE,REPLACE_LOOP_WITH_PIPELINE,REPLACE_VARIABLE_WITH_ATTRIBUTE,SPLIT_ATTRIBUTE,SPLIT_CONDITIONAL,SPLIT_OPERATION,SPLIT_PARAMETER,SPLIT_VARIABLE
11_HASH_CLASSPATH,,,,,,,,,,,,,,,,,,,,,
0076afb3825bd4d3b7ab77310855c2cc4a05813e/value/src/main/java/com/google/auto/value/processor/AutoValueOrOneOfProcessor.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0076afb3825bd4d3b7ab77310855c2cc4a05813e/value/src/main/java/com/google/auto/value/processor/AutoValueOrOneOfTemplateVars.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0076afb3825bd4d3b7ab77310855c2cc4a05813e/value/src/main/java/com/google/auto/value/processor/AutoValueProcessor.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0076afb3825bd4d3b7ab77310855c2cc4a05813e/value/src/main/java/com/google/auto/value/processor/AutoValueTemplateVars.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0076afb3825bd4d3b7ab77310855c2cc4a05813e/value/src/main/java/com/google/auto/value/processor/BuilderSpec.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff523aeab1ec8f5f3ad821e2d75bd3d336dd8096/factory/src/main/java/com/google/auto/factory/processor/FactoryDescriptor.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ff523aeab1ec8f5f3ad821e2d75bd3d336dd8096/factory/src/main/java/com/google/auto/factory/processor/FactoryWriter.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ff523aeab1ec8f5f3ad821e2d75bd3d336dd8096/factory/src/main/java/com/google/auto/factory/processor/Key.java,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
tabela_completa = pd.merge(df, pivoted, right_on="11_HASH_CLASSPATH", left_on="11_HASH_CLASSPATH")
tabela_completa.to_csv(FILE_NAME_TABELA_COMPLETA, index=False)
tabela_completa

,11_HASH_CLASSPATH,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu,DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_1_5,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_5_10,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_10_15,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_15,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_1_5,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_5_10,...,REORDER_PARAMETER,REPLACE_ANONYMOUS_WITH_LAMBDA,REPLACE_ATTRIBUTE_WITH_VARIABLE,REPLACE_LOOP_WITH_PIPELINE,REPLACE_VARIABLE_WITH_ATTRIBUTE,SPLIT_ATTRIBUTE,SPLIT_CONDITIONAL,SPLIT_OPERATION,SPLIT_PARAMETER,SPLIT_VARIABLE
0,0c7c4ab7a304173355caf518e451b7b1cd3eadc3/facto...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1439990cc6a01156c9cbaa98a087cdb03a129fca/value...,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,196c8100d43b5212c31a83912d66791c5a64525b/value...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,26f073f8371577da8237631767ad127c41bbb462/commo...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,324470ba28e85482c7b359a3679601583323d9ab/value...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,4dd4cd8450c351731ad144a53e395217533183a7/commo...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,5344acf2525afaee2874e581b0243a580c6d934d/value...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,6c1190df4d2af62adc20c06b4a2eaf5480b5bd4a/value...,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,8f078e0e3e7821ffda93ce0b28f1f9bd7d8cef31/commo...,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,91d5f32c6a749fde31085b99387028726321c2ee/value...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
tabela_completa.drop(columns=['11_HASH_CLASSPATH'], inplace=True)
tabela_completa

,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu,DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_1_5,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_5_10,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_10_15,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_15,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_1_5,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_5_10,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_10_15,...,REORDER_PARAMETER,REPLACE_ANONYMOUS_WITH_LAMBDA,REPLACE_ATTRIBUTE_WITH_VARIABLE,REPLACE_LOOP_WITH_PIPELINE,REPLACE_VARIABLE_WITH_ATTRIBUTE,SPLIT_ATTRIBUTE,SPLIT_CONDITIONAL,SPLIT_OPERATION,SPLIT_PARAMETER,SPLIT_VARIABLE
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
def transforma_valor(valor):
    if valor > 0:
        return 1
    else:
        return valor

# Aplica a função em cada elemento do DataFrame
tabela_completa = tabela_completa.applymap(transforma_valor)

C:\Users\kleit\AppData\Local\Temp\ipykernel_17308\1884997101.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tabela_completa = tabela_completa.applymap(transforma_valor)


In [88]:
tabela_completa

,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu,DIFERENCA_QNT_ISBLOCKCOMMENT_Manteve,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_1_5,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_5_10,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_10_15,DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_15,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_1_5,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_5_10,DIFERENCA_QNT_ISBLOCKCOMMENT_Diminuiu_10_15,...,REORDER_PARAMETER,REPLACE_ANONYMOUS_WITH_LAMBDA,REPLACE_ATTRIBUTE_WITH_VARIABLE,REPLACE_LOOP_WITH_PIPELINE,REPLACE_VARIABLE_WITH_ATTRIBUTE,SPLIT_ATTRIBUTE,SPLIT_CONDITIONAL,SPLIT_OPERATION,SPLIT_PARAMETER,SPLIT_VARIABLE
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
itemsets = apriori(tabela_completa, min_support=0.1, use_colnames=True, max_len=2)


c:\Python312\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


# Regras

In [90]:

rules = association_rules(itemsets, metric="confidence", min_threshold=0.01)
rules.to_csv(FILE_NAME_TABELA_REGRAS, index=False)

In [91]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou),(DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_1_5),0.782609,0.695652,0.695652,0.888889,1.277778,0.151229,2.739130,1.000000
1,(DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou_1_5),(DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou),0.695652,0.782609,0.695652,1.000000,1.277778,0.151229,inf,0.714286
2,(DIFERENCA_QTN_ISLINECOMMENT_Aumentou),(DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou),0.391304,0.782609,0.347826,0.888889,1.135802,0.041588,1.956522,0.196429
3,(DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou),(DIFERENCA_QTN_ISLINECOMMENT_Aumentou),0.782609,0.391304,0.347826,0.444444,1.135802,0.041588,1.095652,0.550000
4,(DIFERENCA_QTN_ISLINECOMMENT_Manteve),(DIFERENCA_QNT_ISBLOCKCOMMENT_Aumentou),0.521739,0.782609,0.391304,0.750000,0.958333,-0.017013,0.869565,-0.083333
...,...,...,...,...,...,...,...,...,...,...
261,(CHANGE_ATTRIBUTE_TYPE),(RENAME_ATTRIBUTE),0.130435,0.173913,0.130435,1.000000,5.750000,0.107750,inf,0.950000
262,(RENAME_VARIABLE),(CHANGE_VARIABLE_TYPE),0.173913,0.173913,0.130435,0.750000,4.312500,0.100189,3.304348,0.929825
263,(CHANGE_VARIABLE_TYPE),(RENAME_VARIABLE),0.173913,0.173913,0.130435,0.750000,4.312500,0.100189,3.304348,0.929825
264,(RENAME_METHOD),(EXTRACT_VARIABLE),0.260870,0.217391,0.130435,0.500000,2.300000,0.073724,1.565217,0.764706
